In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Input, Conv1D, GRU, merge, Dense, Flatten, LSTM,Activation, Dropout,Bidirectional,Reshape,Conv2D,MaxPool2D,Concatenate
from keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import Adam,SGD
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input
from keras.models import Model

# Data

In [ ]:
dir = "../input/quora-insincere-questions-classification/"

Đọc tập train và test từ dataset.

In [ ]:
train = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
train = train.sample(frac = 1)
train.head(20)

Xem qua nhưng câu hỏi có nhãn bằng 1

In [ ]:
train[train.target == 1]

In [ ]:
test = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
test.head(20)

In [ ]:
sample_sub = pd.read_csv("../input/quora-insincere-questions-classification/sample_submission.csv")
print(sample_sub[sample_sub.prediction <1 ])

In [ ]:
sample_sub.shape

## Pre-processing

# Remove stopword, lowercase, remove punctual, lematize

In [ ]:
from gensim.utils import simple_preprocess 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english')) 
wordnet_lemmatizer = WordNetLemmatizer()
def preprocessing(corpus):
    res = []
    for doc in corpus:
        words = []
        for word in simple_preprocess(doc):
            if word not in stop_words:
                word1 = wordnet_lemmatizer.lemmatize(word, pos = "n")
                word2 = wordnet_lemmatizer.lemmatize(word1,pos = "v")
                word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
                words.append(word3)
                pass
            pass
        res.append(' '.join(words))        
        pass
    return res

In [ ]:
%%time
train['question_text'] = preprocessing(train['question_text'])
test['question_text'] = preprocessing(test['question_text'])

Tính độ dài lớn nhất của câu hỏi trong tập train

In [ ]:
m = 0
for i in train['question_text']:
    m  = max(m, len(i.split()))
    if (len(i.split()) == m):
        print(i, m)
print(m)

Biến đổi từ câu sang vector bằng one hot, mỗi câu tương ứng với một list các số tương ứng với từ

In [ ]:
%%time
vocab_size=150000
max_lenth = 33
train_X = []
for i in range(0, len(train)):
    train_X.append(one_hot(train.iloc[i]['question_text'], vocab_size))
test_X = []
for i in range(0, len(test)):
    test_X.append(one_hot(test.iloc[i]['question_text'], vocab_size))

Load các vector word embedding từ GloVe

In [ ]:
embeddings_index = dict()
f = open('/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
tokenizer = Tokenizer(num_words=150000)
tokenizer.fit_on_texts(train['question_text'])

Tạo embedding matrix

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

cnt = 0
embedding_matrix = np.zeros((vocab_size, 50))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        cnt = cnt + 1
print(vocab_size, ' ', cnt)

In [ ]:
train_X[0]

Padding để tạo thành các chuỗi có độ dài bằng nhau, thuận tiện cho mô hình

In [ ]:
train_X = sequence.pad_sequences(train_X, maxlen=max_lenth, padding='post')
test_X = sequence.pad_sequences(test_X, maxlen=max_lenth, padding='post')

train_Y = train['target']

## Metrics

2 Metric chủ yếu : accuracy, F1

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


## Bi - GRU

Mô hình với Bidirectional GRU với embeddings 


In [ ]:
_input = Input(max_lenth)
embed = Embedding(input_dim=vocab_size, output_dim=50, weights=[embedding_matrix], trainable=True, input_length=max_lenth)(_input)
layer = Bidirectional(GRU(128, return_sequences=True))(embed)
layer = Dense(128, activation = 'relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(64, activation = 'relu')(layer)
layer = Dropout(0.5)(layer)
layer = Flatten()(layer)
prediction = Dense(1, activation = 'sigmoid')(layer)

model = Model(inputs=_input,outputs=prediction)
model.compile(loss=keras.losses.BinaryCrossentropy(),optimizer=Adam(lr=0.0003),metrics = ['accuracy',f1_m])
model.summary()

In [ ]:
history = model.fit(train_X, train['target'], epochs = 10, validation_split = 0.2, verbose = 2, batch_size = 128)

In [ ]:
predict = model.predict(test_X)
predict[:20]

In [ ]:
test['prediction'] = (predict > 0.35).astype(int)
results = test[['qid', 'prediction']]
results.to_csv('submission.csv', index=False)
results.shape

In [ ]:
test.head(20)